In [50]:
#Modified from ChatGPT standard binary tree class

class Node:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.parent = None
        self.reversed = False
        self.netmin = 0
        self.netcost = 0
        self.grosscost = 0
        self.grossmin = 0
        self.rightness = 0
        self.head = None
        self.leftest = 0
        self.rightest = 0
        self.tail = None
        self.height = 0

class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, key):
        if not self.root:
            self.root = Node(key)
        else:
            self._insert_recursive(self.root, key)

    def _insert_recursive(self, node, key):
        if key < node.key:
            if node.left is None:
                node.left = Node(key)
                node.left.parent = node
                node.left.height = node.height + 1
                node.left.rightness = node.rightness - 1
                node.leftest = node.rightness
                cur_node = node
                while cur_node != None:
                    if node.left.rightness < cur_node.leftest:
                        cur_node.leftest = node.left.rightness
                        cur_node.head = node.left
                    cur_node = cur_node.parent
                # if node.parent != None:
                #     node.parent.head = node.left
                # while cur_node != None and node.left.height >= cur_node.height + 2:
                #     cur_node.head = node.left
                #     cur_node = cur_node.left
                cur_node = node
                while cur_node != None:
                    if cur_node.right == None and cur_node.left == None:
                        pass
                    elif cur_node.right == None:
                        cur_node.grosscost = cur_node.left.key
                    elif cur_node.left == None:
                        cur_node.grosscost = cur_node.right.key
                    else:
                        cur_node.grosscost = min(cur_node.left.key, cur_node.right.key)
                    if cur_node.right == None and cur_node.left == None:
                        pass
                    elif cur_node.right == None:
                        cur_node.grossmin = cur_node.left.grosscost
                    elif cur_node.left == None:
                        cur_node.grossmin = cur_node.right.grosscost
                    else:
                        cur_node.grossmin = min(cur_node.left.grosscost, cur_node.right.grosscost)
                    #cur_node.grossmin = min(cur_node.left.grosscost, cur_node.right.grosscost)
                    cur_node.netcost = cur_node.grosscost - cur_node.grossmin
                    cur_node = cur_node.parent
                cur_node = self.root
                cur_node.netmin = cur_node.grossmin
                cur_node = cur_node.left
                while cur_node.left != None:
                    cur_node.netmin = cur_node.grossmin - cur_node.parent.grossmin
                    cur_node = cur_node.left
                                                                                                                       
            else:
                self._insert_recursive(node.left, key)
        elif key > node.key:
            if node.right is None:
                node.right = Node(key)
                node.right.parent = node
                node.right.height = node.height + 1
                node.right.rightness = node.rightness + 1
                node.rightest = node.right.rightness
                cur_node = node
                while cur_node != None:
                    if node.right.rightness > cur_node.rightest:
                        cur_node.rightest = node.right.rightness
                        cur_node.tail = node.right
                    cur_node = cur_node.parent
                # cur_node = self.root
                # if node.parent != None:
                #     node.parent.tail = node.right
                # while cur_node != None and node.right.height >= cur_node.height + 2:
                #     cur_node.tail = node.right
                #     cur_node = cur_node.right
                while cur_node != None:
                    if cur_node.right == None and cur_node.left == None:
                        pass
                    elif cur_node.right == None: 
                        cur_node.grosscost = cur_node.left.key
                    elif cur_node.left == None:
                        cur_node.grosscost = cur_node.right.key
                    else:
                        cur_node.grosscost = min(cur_node.left.key, cur_node.right.key)
                    #cur_node.grosscost = min(cur_node.left.key, cur_node.right.key)
                    if cur_node.right == None and cur_node.left == None:
                        pass
                    elif cur_node.right == None:
                        cur_node.grossmin = cur_node.left.grosscost
                    elif cur_node.left == None:
                        cur_node.grossmin = cur_node.right.grosscost
                    else:
                        cur_node.grossmin = min(cur_node.left.grosscost, cur_node.right.grosscost)
                    
                    #cur_node.grossmin = min(cur_node.left.grosscost, cur_node.right.grosscost)
                    cur_node.netcost = cur_node.grosscost - cur_node.grossmin
                    cur_node = cur_node.parent
                cur_node = self.root
                cur_node.netmin = cur_node.grossmin
                cur_node = cur_node.right
                while cur_node.right != None:
                    cur_node.netmin = cur_node.grossmin - cur_node.parent.grossmin
                    cur_node = cur_node.right
            else:
                self._insert_recursive(node.right, key)
        else:
            # Duplicate keys are not allowed in this simple implementation
            raise ValueError("Duplicate keys are not allowed")

    def inorder_traversal(self):
        result = []
        self._inorder_traversal_recursive(self.root, result)
        return result

    def _inorder_traversal_recursive(self, node, result):
        if node:
            self._inorder_traversal_recursive(node.left, result)
            result.append(node.key)
            right_key = None if node.right == None else node.right.key
            left_key = None if node.left == None else node.left.key
            parent_key = None if node.parent == None else node.parent.key
            head_key = None if node.head == None else node.head.key
            tail_key = None if node.tail == None else node.tail.key
            #leftest_key = None if node.left == None else node.leftest.key
            print({'key': node.key, 'leftest': node.leftest, 'left child': left_key, 'right child': right_key, 'parent' : parent_key, 'reversed': node.reversed, 'netmin': node.netmin, 'netcost': node.netcost, 'grosscost': node.grosscost, 'grossmin': node.grossmin, 'head': head_key, 'tail': tail_key, 'height': node.height})
            #result.append([node.key, node.left, node.right, node.parent, node.reversed, node.netmin, node.netcost, node.grosscost, node.grossmin, node.head, node.tail, node.height])
            self._inorder_traversal_recursive(node.right, result)

# Example Usage:
if __name__ == "__main__":
    tree = BinaryTree()
    keys = [3, 5, 7, 2, 4, 84, -0.2, 6, 19, -2, 8, 7.5, 85]

    for key in keys:
        tree.insert(key)

    print("Inorder Traversal:", tree.inorder_traversal())


{'key': -2, 'leftest': 0, 'left child': None, 'right child': None, 'parent': -0.2, 'reversed': False, 'netmin': 0, 'netcost': 0, 'grosscost': 0, 'grossmin': 0, 'head': None, 'tail': None, 'height': 3}
{'key': -0.2, 'leftest': -3, 'left child': -2, 'right child': None, 'parent': 2, 'reversed': False, 'netmin': 2, 'netcost': -2, 'grosscost': -2, 'grossmin': 0, 'head': -2, 'tail': None, 'height': 2}
{'key': 2, 'leftest': -3, 'left child': -0.2, 'right child': None, 'parent': 3, 'reversed': False, 'netmin': -1.8, 'netcost': 1.8, 'grosscost': -0.2, 'grossmin': -2, 'head': -2, 'tail': None, 'height': 1}
{'key': 3, 'leftest': -3, 'left child': 2, 'right child': 5, 'parent': None, 'reversed': False, 'netmin': -0.2, 'netcost': 2.2, 'grosscost': 2, 'grossmin': -0.2, 'head': -2, 'tail': 85, 'height': 0}
{'key': 4, 'leftest': 0, 'left child': None, 'right child': None, 'parent': 5, 'reversed': False, 'netmin': 0, 'netcost': 0, 'grosscost': 0, 'grossmin': 0, 'head': None, 'tail': None, 'height': 2}

In [ ]:
def path(v):
    w = v
    while w.parent != None:
        w = w.parent
    return w

def head(p):
    if p.reversed:
        return p.tail
    return p.head

def before(v):
    nodes = []
    w = v
    deepest_right_child = v
    while w.parent != None:
        nodes.append(w.parent)
        w = w.parent
    for i in range(len(nodes) - 1, 1, -1):
        if nodes[i].right == nodes[i-1]:
            deepest_right_child = nodes[i-1]
    u = deepest_right_child.parent.left
    if u.left == None and u.right == None:
        return u
    if u.reversed:
        return u.head
    return u.tail

def after(v):
    nodes = []
    w = v
    deepest_left_child = v
    while w.parent != None:
        nodes.append(w.parent)
        w = w.parent
    for i in range(len(nodes) - 1, 1, -1):
        if nodes[i].left == nodes[i-1]:
            deepest_left_child = nodes[i-1]
    u = deepest_left_child.parent.right 
    if u.reversed:
        return u.head
    return u.tail

def cost(v):
    nodes = []
    w = v
    deepest_left_child = v
    while w.parent != None:
        nodes.append(w.parent)
        w = w.parent
    for i in range(len(nodes) - 1, 1, -1):
        if nodes[i].left == nodes[i-1]:
            deepest_left_child = nodes[i-1]
    return deepest_left_child.parent.grosscost
    
def tail(p):
    if p.reversed:
        return p.head
    return p.tail
    
def reverse(p):
    p.reversed = not p.reversed
    return p
    
def update(p, x):
    p.netmin += x
    return p
    
def mincost(p):
    u = p
    while True:
        if (u.netcost == 0 and (u.right.netmin > 0 or (u.right.right == None and u.right.left == None))):
            break
        elif u.right.netcost == 0 and (u.right.right != None or u.right.left != None):
            u = u.right
        elif u.netcost > 0:
            u = u.left
    if u.left == None and u.right == None:
        return u
    if u.reversed:
        return u.head
    return u.tail
        
def construct(v, w, x):
    root = Node()
    root.grosscost = x
    root.left = v
    root.left = w
    return root

def destroy(u):
    v = u.left
    w = u.right
    x = u.grosscost
    del u
    return [v, w, x]

In [ ]:
def split(v):
    w = path(v)
    head_v = head(w)
    tail_v = tail(w)
    before_v = before(v)
    after_v = after(v)
    x = before_v.key
    y = v.key
    p = 
    q = 
    if v == w:
        
    return [p, q, x, y]

def splice(p):
    v = tail(p).parent
    